In [21]:
from itertools import permutations

In [22]:
def leitura_fasta_q(fasta_q):
    'extrai as sequencias de um arquivo fastaq'
    reads = []  
    with open (fasta_q) as fq:     
        while True:
            fq.readline() #skip line
            sequencia =  fq.readline().rstrip()
            fq.readline() #skip
            qualidade= fq.readline().rstrip()
            if len(sequencia) == 0 or len(qualidade) == 0:
                break
            reads.append(sequencia)
    return reads

In [23]:
'''Dada uma sequencia e um trecho de interesse, encontrar os hits na sequencia e 
 e calcular as distancias entre os hits e o trecho de interesse'''
import bisect
   
class SubseqIndex(object):
    
    TOTAL_SUBSEQS = 2
    SUBSEQS = {0: 'first', 1:'second'}

    def __init__(self, t, k, ival):
        """ival: intervalo entre os caracteres do texto 
           k: quantidade de caracteres na chave do indice
           t: texto
        Transforma um texto em um indice, cujos elementos sao subsequencias do tamanho (k)
        e extraidas de forma sequencial, respeitando o espaco  definido  entre os caracteres (ival).
        Permite apenas realizar buscas exatas.
        Ex: SubseqIndex("ATAT", 2, 2)  retorna indice ("AA", 0) and ("TT", 1)"""
        self.k = k  # num characters per subsequence 
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)

        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        '''p: padrao buscado
        Obtem os indices encontrados para determinada busca. Para isso,
        extrai subsequencia do trecho buscado considerando o mesmo padrao que foi usado 
        para a montagem do indice. Em seguida, faz a busca no indice. Se nao encontrar, 
        faz uma nova tentativa, porem começando a subsequencia do segundo caracter. 
        Retorna os hits e se foi na primeira ou na segunda subsequencia '''
        hits = []
        
        for i in range (self.TOTAL_SUBSEQS):
            
            subseq = p[i:i+self.span:self.ival]  # query with  subseq
           # print('subseq ', self.SUBSEQS[i], ':' , subseq)
            
            j = bisect.bisect_left(self.index, (subseq, -1))  # binary search
            
            while j < len(self.index):  # collect matching index entries
                if self.index[j][0] != subseq:
                    break
                hits.append([self.index[j][1],  subseq]) #[pos, first or second subseq]
                j += 1
                            
            if  len(hits) > 0:
                break
                
       # print('hits',len(hits))
        return hits


In [24]:
def find_overlap(sufix,prefix,n):
    '''encontra overlaps de tamanho n entre o sufixo e o prefixo 
       Ex -  overlap de tamanho 4:
           CGCGAAGT (sufixo)
               AAGTCCCCCAAA (prefixo)
    '''
    start = 0
    while True:
        start =  sufix.find(prefix[:n], start)
        
        if start == -1: # nao encontrou overlap de tamanho n
            return 0
        
        if prefix.startswith(sufix[start:]):
            return len(sufix) - start
        
        start += 1

In [25]:
def find_all_overlaps (reads, n):
    '''Obtem um conjunto de reads, permuta as reads par a par, verificando o overlap entre elas'''
    
    overlaps = {}
    permuts =  []
    
    for sufix, prefix in permutations (reads, 2):
        permuts.append([sufix, prefix])
    for pair in permuts:
        print('sufix', pair[0], 'prefix', pair[1])
        overlap = find_overlap(pair[0], pair[1], n)
        if overlap > 0:
            overlaps[(pair[0], pair[1])] = overlap
    return overlaps

In [26]:
#obtem as sequencias do fastaq
reads =  leitura_fasta_q('ERR266411_1.for_asm.fastq')
all_reads = ''.join(reads)

In [27]:
#para cada read, monta um indice de k-mers de tamanho n
read_hits = {}
span = 1
n = 6

#cria um dicionario a partir de todas as reads lidas tendo cada kmer=6
index = SubseqIndex(all_reads, n, span)

In [28]:
#para cada read busca os hits no indice e adiciona ao dicionario seq_hits
#seq_hits = {}
#for read in reads:
#    hits = index.query(read)
#    seq_hits[read] = hits[0]

In [29]:
#para cada read busca os hits no indice e adiciona ao dicionario seq_hits
kmer_reads = {}

#associa as reads a cada kmer
for read in reads:
    hits = index.query(read)
    
    # inicializa a key no dict, se nao existir
    if  hits[0][1] not in kmer_reads.values():
        kmer_reads[hits[0][1]] = []
        
    #adiciona a read    
    kmer_reads[hits[0][1]].append(read)

    


{'TAAACA': ['TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC'], 'AACAAG': ['AACAAGAGAATCTCTACCATGAACAAAATGTGACTCATATCTAAACCAGTCCTTGATGAACGTGCCAAGCATATTAAGCCACTTCTCCTCATCCAACGCG'], 'AGCCGA': ['AGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGGGTTTTATC'], 'GACAAA': ['GACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGAAGCAGAAGTTAACACTTTCG'], 'CTGTAG': ['CTGTAGCAATCCAAACTTTGTTACTCGTCAGAAAATCGAAATCATCTTCGGTTAAATCCAAAACGGCAGAAGCCTGAATGAGCTTAATAGAGGCCAAAGC'], 'CTGTGA': ['CTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATCGCTTCCATGACGCAGAAGTTAAA'], 'CAAATC': ['CAAATCGGTCTGGAAACGTACGGATTGTTCAGTAACTTGACTCATGATTTCTTACCTATTAGTGGTTGAACAGCATCGGACTCAGATAGTAATCCACGCT'], 'GTAAAC': ['GTAAACGCGAACAATTCAGCGGCTTTAACCGGACGCTGGACGCCATTAATAATGTTTTCCGTAAATTCAGCGCCTTCCATGATGAGACAGGCCGTTTGAA'], 'TTACTG': ['TTACTGAACAATCCGTACGTTTCCAGACCGCTTTGGCCTCTATTAAGCTCATTCAGGCT

In [30]:
#encontra os overlaps de tamanho n
#find_all_overlaps(reads, 6)